Word2Vec

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow as tf
import utils
import word2vec_utils

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

In [3]:
VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128
SKIP_WINDOW = 1
NUM_SAMPLED = 64
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
VISUAL_FLD = "visualization"
SKIP_STEP = 5000


In [4]:
DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000

In [5]:
def word2vec(dataset):
    with tf.name_scope('data'):
        iterator = dataset.make_initializable_iterator()
        center_words, target_words = iterator.get_next()
    
    
    with tf.name_scope('embed'):
        embed_matrix = tf.get_variable('embed_matrix', shape = [VOCAB_SIZE, EMBED_SIZE], initializer = tf.random_uniform_initializer())
        embed = tf.nn.embedding_lookup(embed_matrix, center_words, name = 'embedding')
    
    with tf.name_scope('loss'):
        nce_weight = tf.get_variable('nce_weight', shape = [VOCAB_SIZE, EMBED_SIZE], initializer = tf.truncated_normal_initializer(stddev = 1.0 /(EMBED_SIZE**0.5)))
        nce_bias = tf.get_variable('nce_bias', initializer = tf.zeros([VOCAB_SIZE]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights = nce_weight,biases = nce_bias,labels = target_words, 
                                             inputs = embed,num_sampled = NUM_SAMPLED, num_classes = VOCAB_SIZE, name = 'loss'))
        
    
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
        
    utils.safe_mkdir('Lec4_checkpoints')
    
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer())
        
        total_loss = 0.0
        writer = tf.summary.FileWriter('graphs/word2vec_simple',sess.graph)
        
        for index in range(NUM_TRAIN_STEPS):
            try:
                loss_batch,_ = sess.run([loss, optimizer])
                total_loss += loss_batch
                if (index + 1)% SKIP_STEP == 0:
                    print("Average loss at ste {}:{:5.1f}".format(index, total_loss/SKIP_STEP))
                    total_loss = 0.0
            except tf.errors.OutOfRangeError:
                sess.run(iterator.intializer)
        writer.close()

def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES,VOCAB_SIZE,BATCH_SIZE,SKIP_WINDOW, VISUAL_FLD)
                
        
        
        

In [6]:
dataset = tf.data.Dataset.from_generator(gen,(tf.int32, tf.int32),(tf.TensorShape([BATCH_SIZE]),tf.TensorShape([BATCH_SIZE,1])))
word2vec(dataset)

data/text8.zip already exists
Average loss at ste 4999: 65.4
Average loss at ste 9999: 18.1
Average loss at ste 14999:  9.7
Average loss at ste 19999:  6.7
Average loss at ste 24999:  5.8
Average loss at ste 29999:  5.2
Average loss at ste 34999:  5.0
Average loss at ste 39999:  4.8
Average loss at ste 44999:  4.8
Average loss at ste 49999:  4.7
Average loss at ste 54999:  4.7
Average loss at ste 59999:  4.7
Average loss at ste 64999:  4.6
Average loss at ste 69999:  4.7
Average loss at ste 74999:  4.6
Average loss at ste 79999:  4.7
Average loss at ste 84999:  4.7
Average loss at ste 89999:  4.7
Average loss at ste 94999:  4.6
Average loss at ste 99999:  4.6
